# Item 24: Use `None` and Docstrings to Specif Dynamic Default Arguments

In [1]:
# Some times we need a non-static type as a keyword argument's default value
from time import sleep
from datetime import datetime

def log(message, when=datetime.now()):
    print(f'{when}: {message}')

log('Hi there!')
sleep(0.1)
log('Hi again!')

2021-08-31 22:14:09.045808: Hi there!
2021-08-31 22:14:09.045808: Hi again!


The above code doesn't work as expected. Notice that both timestamps are the same, this is because `datetime.now` is executed only one time: when the function is defined. A default argument value is evauluated only once per module load, which usaully happens when a program starts up. After the module containing this code is loaded, the `datetime.now()` default argument will never be evaluated again. 

The convention for achieving the desired result in Python is to provide a defualt value of None and to document the actual behavior in the docstring. When your code sees the argument value `None`, we allocate the default value accordingly.

In [2]:
def log(message, when=None):
    """Log a message with a timestamp.

    Args:
        message: Message to print
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')

In [3]:
# Now the timestamps will be different
log('Hi there')
sleep(0.1)
log('Hi again')

2021-08-31 22:21:34.876893: Hi there
2021-08-31 22:21:34.980423: Hi again


Using `None` for default argument values is specially important when the arguments are mutable.

In [4]:
# Say we want to load a value encoded as JSON data; if encoding the data fails, I want an empty dictionary
# to be returned by default
import json

def decode(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default

The problem with the code above is that the dictionary specified for `default` will be shared by all calls to `decode` because default argument values are evaluated only once (at module load time). This can cause extremely surprising behavior:

In [5]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)

Foo: {'stuff': 5, 'meep': 1}
Bar: {'stuff': 5, 'meep': 1}


We might expect two different dictionaries, each with a single key and value. But modifying one, also seems to modify the other. The culprit is that `foo` and `bar` are both equal to the `default` parameter. The are the same dictionary object:

In [6]:
assert foo is bar

In [7]:
# The solution to this is to set the keyword argument default value to None and then document the behavior in the
# function's docstring
def decode(data, default=None):
    """Load JSON data from astring.
    
    Args:
        data: JSON data to decode.
        default: Value to return if decoding fails
            Defaults to an empty dictionary.
    """
    try:
        return json.loads(data)
    except ValueError:
        if default is None:
            default = {}
        return default

In [8]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)
assert bar is not foo

Foo: {'stuff': 5}
Bar: {'meep': 1}


This approach also works with type annotations. Here, the `when` argument is marked as having an `Optional` value that is a `datetime`. Thus, only two valid choices for `when` are `None` or a `datetime` object:

In [9]:
from typing import Optional

def log_type(message: str,
             when: Optional[datetime]=None) -> None:
    """Log a message with a timestamp.

    Args:
        message: Message to print
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')